# Convert h5 model to trt model

In [1]:
import sys 
import os

NOMEROFF_NET_DIR = "../../"

# setup result model dirs
SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/saved_model_ocr_eu_ua_2004_2015")
TRT_SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/trt_saved_model_ocr_eu_ua_2004_2015")

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
import tensorflow as tf
import matplotlib.image as mpimg

# dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

from NomeroffNet import  TextDetector

## load latest h5 model

In [3]:
textDetector = TextDetector.get_static_module("eu_ua_2004_2015")()
textDetector.load("latest")

In [4]:
textDetector.MODEL.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_eu_ua_2004_2015 (Inpu [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_eu_ua_2004_2015[0][0]  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_______________________________________________________________________________________

## convert to pb model

In [5]:
tf.saved_model.save(textDetector.MODEL, SAVED_MODEL_DIR)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../.././models/saved_model_ocr_eu_ua_2004_2015/assets


## convert pb model to trt model

In [6]:
params = tf.experimental.tensorrt.ConversionParams(
    minimum_segment_size=50,
    max_workspace_size_bytes=1 << 25,
    precision_mode  = "FP16",
)

converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir=SAVED_MODEL_DIR,
    conversion_params=params)
converter.convert()
converter.save(TRT_SAVED_MODEL_DIR)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Assets written to: ../.././models/trt_saved_model_ocr_eu_ua_2004_2015/assets


## run trt model

In [7]:
saved_model_loaded = tf.saved_model.load(TRT_SAVED_MODEL_DIR)

In [8]:
import matplotlib.image as mpimg
import numpy as np

img_path = '../../examples/crop_np_images/JJF509.png'
imgs = [mpimg.imread(img_path)]
Xs = []
for img in imgs:
    Xs.append(textDetector.normalize_pb(img[:,:,:3]))

In [9]:
f = saved_model_loaded.signatures["serving_default"]

In [10]:
tensorX = tf.convert_to_tensor(np.array(Xs).astype(np.float32))

In [11]:
f(tensorX)["softmax_eu_ua_2004_2015"]

<tf.Tensor: shape=(1, 32, 23), dtype=float32, numpy=
array([[[2.53122905e-03, 7.53675587e-04, 2.40801106e-04, 7.28358806e-04,
         7.87236146e-04, 2.42610011e-04, 7.77694368e-07, 9.33263567e-04,
         2.98372749e-02, 6.13803387e-01, 7.83105788e-04, 5.60349561e-02,
         5.07272547e-03, 3.34981276e-04, 6.95553899e-04, 1.53762630e-06,
         1.80886791e-03, 2.66470253e-01, 7.28129689e-03, 2.33316212e-03,
         7.21380021e-03, 1.79350458e-03, 3.17647035e-04],
        [2.57452712e-07, 6.38040729e-05, 1.13903334e-05, 6.06650156e-06,
         7.10987990e-07, 8.91365926e-06, 1.77526427e-09, 2.21713653e-05,
         2.65006995e-04, 7.44642736e-03, 2.75659114e-01, 4.29416835e-01,
         8.09002260e-04, 1.15487914e-07, 2.74810463e-05, 2.97060634e-08,
         6.83444133e-03, 2.79356509e-01, 2.18927326e-07, 2.72398192e-05,
         2.35889857e-07, 1.06760083e-07, 4.39349460e-05],
        [2.79678163e-13, 4.94551289e-09, 1.70663235e-08, 3.55910718e-11,
         1.14187451e-10, 4.5